TP CSV
===

In [ ]:
import csv
from IPython.display import HTML, display
from operator import itemgetter, attrgetter

def import_csv2(filename, types=None):
    """
    ouvre un fichier et retourne une liste de dictionnaires
    on peut préciser les types pour chaque colonne exemple types=['str','int','float','str'],
    dans ce cas le contenu est casté (changement de type)
    """
    with open(filename, newline='', encoding='utf8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        # Si on a précise les types
        if types != None:
            result=[]
            for row in reader:
                keys = [k for k in row.keys()]
                values = [v for v in row.values()]
                updates = {}
                for i in range(len(types)):
                    if types[i]=='int':
                        updates[keys[i]]=int(values[i])
                    elif types[i]=='float':
                        updates[keys[i]]=float(values[i])
                row.update(updates)
                result.append(row)
            return result
        # Si on n'a pas précisé de type
        else:
            return list(reader)
        
def filter_colums(datas, list_of_column_names, action='keep'):
    """
    Retourne les données filtrées par colonne
    list_of_column_names : la liste des colonnes
    action : defaut = 'keep' sinon 'remove'
    """
    if datas:
        result=[]
        # récupérons les clés
        keys=[k for k in datas[0].keys()]
        for row in datas:
            new_row = row.copy()
            result.append(new_row)
            for k in keys:
                if action=='remove':
                    if k in list_of_column_names:
                        new_row.pop(k)
                else:
                    if k not in list_of_column_names:
                        new_row.pop(k)
        return result
    return datas

def filter_lines(datas, key, value, operator = '='):
    """
    Filtre les lignes dont la valeur de la clé vaut 'value'
    Les opérateurs peuvent être 'in', '<', '>' par défaut =
    """
    if operator == 'in':
         return [row for row in datas if value.lower() in row[key].lower()]
    elif operator == '>':
        if isinstance(datas[0][key], str):
            return [row for row in datas if row[key].lower() > value.lower()]
        else:
            return [row for row in datas if row[key] > value]
    elif operator == '<':
        if isinstance(datas[0][key], str):
            return [row for row in datas if row[key].lower() < value.lower()]
        else:
            return [row for row in datas if row[key] < value]
    else:
        if isinstance(datas[0][key], int):
            return [row for row in datas if row[key] == int(value)]
        elif isinstance(datas[0][key], float):
            return [row for row in datas if row[key] == float(value)]
        else:
            return [row for row in datas if row[key].lower() == value.lower()]

def display_table(datas):
    """
    Retourne le tableau sous forme de tableau html
    """
    if datas:
        display(HTML(
           '<table><thead><tr><th>{}</th></tr></thead><tbody><tr>{}</tr></tbody></table>'.format(
               '</th><th>'.join([k.title() for k in datas[0].keys()]),
               '</tr><tr>'.join(
                   '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row.values())) for row in datas)
               )
        ))
    else:
       display(HTML('Rien a afficher')) 

Importation des données depuis le fichier 'datas.csv'
---

In [ ]:
from os import path
if not path.exists('datas.csv'):
    import zipfile
    with zipfile.ZipFile('datas.zip', 'r') as zip_ref:
    zip_ref.extractall('.')    

dicos = import_csv2('datas.csv', types=['str','str','str','str','int'])

Tous les titres de Madonna entre 2000 et 2008. Colonnes affichées Titre, artiste, année.
---

In [ ]:
f = filter_lines(dicos, 'artist_name', 'madonna')
f = filter_lines(f, 'year', 2000, operator = '>')
f = filter_lines(f, 'year', 2008, operator = '<')
f = filter_colums(f, ['song_id','release'], action='remove')
display_table(f)

Les trier par année puis par titre

In [ ]:
f = sorted(f, key=itemgetter('year','title'))
display_table(f)

Trouver tous les morceaux de l'artiste 'Tricky' de l'album 'blowback'.
---

In [ ]:
f = filter_lines(dicos, 'artist_name', 'tricky')
f = filter_lines(f, 'release', 'blowback')
display_table(f)


Trouver tous les morceaux dont les artistes commencent par 'ab' et dont l'année est > à 2000
---

In [ ]:
f = filter_lines(dicos, 'artist_name', 'ab', operator='>')
f = filter_lines(f, 'artist_name', 'ac', operator='<')
f = filter_lines(f, 'year', 2005, operator = '>')
display_table(f)

les trier par nom d'artiste dans l'ordre descendant.

In [ ]:
f=sorted(f, key=itemgetter('artist_name'), reverse=True)
display_table(f)